In [1]:
import random
import unittest
from ddt import ddt, data, unpack

import sys
sys.path.append('../Environments/')
from ColoredGridWorld import SetUpInferenceSpace as targetCode

In [14]:
@ddt
class TestInferenceSpaceSetup(unittest.TestCase):
    def setUp(self): 
        #setUp for envPolicySpace specifically
        self.dimensions = (3,3)
        self.stateSpace = {(0,0): 'white', (0,1): 'white', (0,2): 'white', (1,0): 'white', (1,1): 'white', (1,2):'white', (2,0): 'blue', (2,1):'red', (2,2):'green'}
        self.actions = [(-1,0),(0,1),(0,-1),(1,0)]
        self.envSpace = [(targetCode.HashableWorld({'white':0,'green':2,'red':-10,'blue':0}, True),(2,2)), (targetCode.HashableWorld({'white':0,'green':0,'red':-10,'blue':2}, True),(2,0)), (targetCode.HashableWorld({'white':0,'green':2,'red':-10,'blue':-10}, True),(2,2)), (targetCode.HashableWorld({'white':0,'green':-10,'red':-10,'blue':2}, True),(2,0))]
        self.hyperparameters = (10e-7, .9, 20, 0.01)

    # buildWorld #######################################################################################################
    #Function to test both that the world is indeed hashable(can form a dictionary's key) and also checks its custom eq function
    @data(({'red':-10, 'white':0}, True), ({'hello':'a', 'world':'b'}, False))
    @unpack
    def test_World_checkHashingAndEquals(self, colourReward, isDeterministic):
        world1 = targetCode.HashableWorld(colourReward, isDeterministic)
        world2 = targetCode.HashableWorld(colourReward, isDeterministic)
        dictWorld1 = {world1:1}
        dictWorld2 = {world2:5}
        self.assertEqual(dictWorld1.keys(),dictWorld2.keys())

    # getUtilitySpace #######################################################################################################
    @data((['orange'],[0,-2],{'yellow':10}), (['orange'],[0,-2],{'yellow':10, 'white':0}), (['orange', 'blue'], [0,-2], {'yellow':10}), (['orange', 'blue', 'purple'], [0,5,-2], {'yellow':10, 'white':0}))
    @unpack
    def test_utilitySpace_checkSizeOfUtilitySpace(self, variableColours, variableReward, constantRewardDict):
        # make sure the the size of space is correct
        utilitySpace = targetCode.buildUtilitySpace(variableColours, variableReward, constantRewardDict)
        self.assertEqual(len(variableReward)**len(variableColours), len(utilitySpace))
    
    #simple example with manually formed utilityExample for a particular space
    @data((['orange', 'blue'],[0,-2, 5],{'yellow':10}))
    @unpack
    def test_utilitySpace_itemInUtilitySpace(self, variableColours, variableReward, constantRewardDict):
        utilitySpace = targetCode.buildUtilitySpace(variableColours, variableReward, constantRewardDict)
        utilityExample = {'orange':-2, 'yellow':10, 'blue':5}
        self.assertIn(utilityExample, utilitySpace)
    
    #Checking for a randomly formed utilityExample in multiple spaces
    @data((['orange'],[0,-2],{'yellow':10}),(['orange', 'blue', 'purple'], [-2,8], {'red':0}), (['orange', 'blue', 'purple', 'red'], [0,5,-2,-5,-10], {'yellow':10, 'white':0}))
    @unpack
    def test_utilitySpace_itemInUtilitySpace(self, variableColours, variableReward, constantRewardDict):
        utilitySpace = targetCode.buildUtilitySpace(variableColours, variableReward, constantRewardDict)
        utilityExample = {colour:random.choice(variableReward) for colour in variableColours}
        utilityExample.update(constantRewardDict)
        self.assertIn(utilityExample, utilitySpace)
        

    # getWorldSpace #######################################################################################################
    @data(([{'yellow':10,'purple':-2},{'yellow':10}],[True]), ([{'yellow':10}, {'white':0, 'blue':0}, {'orange':-2},{'purple':-5}], [True, False]))
    @unpack
    def test_WorldSpace_checkSizeOfWorldSpace(self, utilitySpace, transitionSpace):
        # make sure the the size of space is correct
        worldSpace = targetCode.buildWorldSpace(utilitySpace, transitionSpace)
        self.assertEqual(len(utilitySpace)*len(transitionSpace), len(worldSpace))
    
    #simple example with manually formed worldExample for a particular space
    @data(([{'yellow':10}, {'white':0}, {'orange':-2,'blue':0},{'purple':-5}], [True, False]))
    @unpack
    def test_WorldSpace_itemInWorldSpace(self, utilitySpace, transitionSpace):
        worldSpace = targetCode.buildWorldSpace(utilitySpace, transitionSpace)
        worldExample = targetCode.HashableWorld({'white':0},False)
        self.assertIn(worldExample, worldSpace)

    #Checking for a randomly formed worldExample in multiple spaces
    @data(([{'yellow':10}, {'white':0}, {'orange':-2,'blue':0},{'purple':-5}], [True,False]), ([{'yellow':10,'purple':-2},{'yellow':10}],[True]), ([{'orange':-4}, {'yellow':10,'purple':-2,'blue':5}],[True,False]))
    @unpack
    def test_WorldSpace_itemInWorldSpace(self, utilitySpace, transitionSpace):
        worldSpace = targetCode.buildWorldSpace(utilitySpace, transitionSpace)
        worldExample = targetCode.HashableWorld(random.choice(utilitySpace),random.choice(transitionSpace))
        self.assertIn(worldExample, worldSpace)

    # getEnvironmentPolicySpace #######################################################################################################
    
    #@data(((2,2),{(0,0):'white', (0,1): 'white', (1,0): 'white', (1,1): 'orange'}, [(1,0), (-1,0), (0,1), (0,-1)], [ (targetCode.HashableWorld({'white':0, 'orange':2}, True), (1,1)), (targetCode.HashableWorld({'white':0, 'orange':0}, True), (1,1)), (targetCode.HashableWorld({'white':0, 'orange':2}, True), (1,0)), (targetCode.HashableWorld({'white':0, 'orange':0}, True), (1,0))], (10e-7, 0.94, 20, 0.01)))
    #@unpack
    def test_envPolicySpace_checkSizeOfEnvPolicySpace(self):
        # make sure the the size of space is correct
        envPolicySpace = targetCode.buildEnvPolicySpace(self.dimensions, self.stateSpace, self.actions, self.envSpace, self.hyperparameters)
        self.assertEqual(len(self.envSpace), len(envPolicySpace))
        
    @data((targetCode.HashableWorld({'white':0,'green':2,'red':-10,'blue':0}, True),(2,2)), (targetCode.HashableWorld({'white':0,'green':0,'red':-10,'blue':2}, True),(2,0)))
    def test_envPolicySpace_itemInEnvPolicySpace(self, env):
        envPolicySpace = targetCode.buildEnvPolicySpace(self.dimensions, self.stateSpace, self.actions, self.envSpace, self.hyperparameters)
        self.assertIn(env,envPolicySpace.keys())
    
    #to check that the policy of the given environment maps to its own goal
    @data((targetCode.HashableWorld({'white':0,'green':2,'red':-10,'blue':0}, True),(2,2)), (targetCode.HashableWorld({'white':0,'green':0,'red':-10,'blue':2}, True),(2,0)))
    def test_envPolicySpace_checkPolicyMapsToEnv(self, env):
        envPolicySpace = targetCode.buildEnvPolicySpace(self.dimensions, self.stateSpace, self.actions, self.envSpace, self.hyperparameters)
        TerminalState = env[1]
        envPolicy = envPolicySpace[env][1]
        isTerminalStateInPolicy = TerminalState in envPolicy.keys()
        self.assertFalse(isTerminalStateInPolicy)
    
    def tearDown(self):
        pass

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...................
----------------------------------------------------------------------
Ran 19 tests in 0.046s

OK
